### Imports

In [ ]:
import json
import os

from huggingface_hub import InferenceClient
from langchain import hub
from langchain.agents import (AgentExecutor, create_react_agent,
                              initialize_agent, AgentType)
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import Tool
from langchain_experimental.tools import PythonREPLTool
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_google_community import GoogleSearchAPIWrapper

In [29]:
with open('creds.json') as file:
    creds = json.load(file)

os.environ["OPENAI_API_KEY"] = creds["OPENAI_API_KEY"]
os.environ["HF_TOKEN"] = creds["HUGGINGFACEHUB_API_TOKEN"]
os.environ["GOOGLE_API_KEY"] = creds["GOOGLE_API_KEY"]
os.environ["GOOGLE_CSE_ID"] = creds["GOOGLE_CSE_ID"]


### Завдання 1: Виклик LLM з базовим промптом

Створіть можливість викликати LLM зі звичайним текстовим промптом.

Промпт має дозвляти отримати інформацію простою мовою на певну тему. В цьому завданні ми хочемо дізнатись про тему "Квантові обчислення".

Відповідь моделі повинна містити визначення, ключові переваги та поточні дослідження в цій галузі.

Обмежте відповідь до 200 символів і пропишіть в промпті аби відповідь була короткою (це зекономить Вам час і гроші на згенеровані токени).

В якості LLM можна скористатись як моделлю з HugginFace (рекомендую Mistral), так і ChatGPT4 або ChatGPT3. В обох випадках треба імпортувати потрібну "обгортку" (тобто клас, який дозволить ініціювати модель) з LangChain для виклику LLM за API, а також зчитати особистий токен з файла, наприклад, `creds.json`, який розміщений у Вас локально і Ви НЕ здаєте його в ДЗ і НЕ комітите в git 😏

Встановіть своє значення температури на свій розсуд (тут немає правильного чи неправильного значення) і напишіть, чому ви обрали саме таке значення для цього завдання.  

Запити можна робити як українською, так і англійською - орієнтуйтесь на те, де і чи хочете ви потім лишити цей проєкт і відповідна яка мова буде пасувати більше. В розвʼязках промпти - українською.

In [3]:
overal_temperature = 0.1

For this task the responses must be correct and precise. This suggests using a lower  temperature to reduce randomness in the model's output.

In [15]:
mistral = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    temperature=overal_temperature,
    max_new_tokens=200
)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
request = ("Give a definition of quantum computing, key benefits, "
           "and current research in this field. The answer must be very brief.")

print(mistral.invoke(request))

d:\anaconda3\envs\py312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)




Quantum computing is a technology that utilizes quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. Key benefits include the potential for exponentially faster solutions to certain problems compared to classical computers, and the ability to perform multiple calculations simultaneously. Current research focuses on developing practical quantum algorithms, building scalable quantum hardware, and improving error correction techniques.


To avoid the **FutureWarning** abour `post` being deprecated, we should use `InferenceClient` instead of `HuggingFaceEndpoint`.

In [ ]:
client = InferenceClient(
    provider='hf-inference'
)

messages = [
    {
        "role": "user",
        "content": ("Give a definition of quantum computing, key benefits, "
                    "and current research in this field. "
                    "The answer must be very brief.")
    }
]

output = client.chat_completion(
    model="mistralai/Mistral-7B-Instruct-v0.3",
    messages=messages,
    max_tokens=200,
    temperature=overal_temperature
)

print(output.choices[0].message.content)

Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. Key benefits include the potential for significantly faster computation and improved efficiency for certain complex problems, such as factoring large numbers and simulating quantum systems. Current research focuses on developing practical quantum algorithms, building scalable quantum hardware, and improving error correction techniques.


### Завдання 2: Створення параметризованого промпта для генерації тексту
Тепер ми хочемо оновити попередній фукнціонал так, аби в промпт ми могли передавати тему як параметр. Для цього скористайтесь `PromptTemplate` з `langchain` і реалізуйте параметризований промпт та виклик моделі з ним.

Запустіть оновлений функціонал (промпт + модел) для пояснень про теми
- "Баєсівські методи в машинному навчанні"
- "Трансформери в машинному навчанні"
- "Explainable AI"

Виведіть результати відпрацювання моделі на екран.

In [ ]:
prompt = PromptTemplate(
    input_variables=['theme'],
    template=("Give a definition of {theme}, key benefits, "
              "and current research in this field. The answer must be very brief.")
)

In [23]:
for theme in ['Bayesian methods in Machine Learning',
              'Transformers in Machine Learning',
              'Explainable AI']:
    print(mistral.invoke(prompt.format(theme=theme)))

d:\anaconda3\envs\py312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)




Bayesian methods in Machine Learning are a set of statistical techniques that update the probability of a hypothesis based on observed data. They use Bayes' theorem to calculate the posterior probability, which represents the updated belief about the hypothesis given the data.

Key benefits include:
1. Incorporation of prior knowledge: Bayesian methods allow for the use of prior knowledge or beliefs about the problem domain, which can improve model performance.
2. Probabilistic predictions: Bayesian methods provide a measure of uncertainty in predictions, which can be useful in decision-making processes.
3. Flexibility: Bayesian methods can be applied to a wide range of problems, including classification, regression, and clustering.

Current research in this field focuses on developing more efficient algorithms, incorporating complex prior distributions, and applying Bayesian methods to deep learning and other advanced machine learning techniques.


d:\anaconda3\envs\py312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)




Transformers in Machine Learning are a type of model architecture that uses self-attention mechanisms to process input data. They are key components in models like BERT and GPT-3, and are known for their ability to handle long-range dependencies in data. Benefits include improved performance on tasks like language modeling and translation, and the ability to capture complex relationships between words. Current research focuses on improving efficiency, reducing training time, and applying Transformers to new domains like computer vision and music.


d:\anaconda3\envs\py312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)




Explainable AI (XAI) refers to AI systems that can provide clear, understandable explanations for their decisions. Key benefits include increased trust, transparency, and accountability, as well as the ability to identify and correct biases. Current research focuses on developing methods for generating interpretable models, visualizing complex decision-making processes, and developing standards for evaluating explainability.


In [ ]:
prompt = ("Give a definition of {theme}, key benefits, "
          "and current research in this field. "
          "The answer must be very brief and keep the concise.")

chat_template = ChatPromptTemplate.from_messages(
    [('human', prompt)]
)

for theme in ['Bayesian methods in Machine Learning',
              'Transformers in Machine Learning',
              'Explainable AI']:
    messages = [
        {"role": "user",
         "content": chat_template.format_messages(theme=theme)[0].content}
    ]
    output = client.chat_completion(
        model="mistralai/Mistral-7B-Instruct-v0.3",
        messages=messages,
        max_tokens=200,
        temperature=overal_temperature
    )

    print(output.choices[0].message.content)

Bayesian methods in Machine Learning are a set of statistical techniques that update the probability of a hypothesis based on observed data, using Bayes' theorem. They allow for the incorporation of prior knowledge and provide a natural way to handle uncertainty. Key benefits include:

1. Incorporation of prior knowledge
2. Handling of uncertainty through probabilistic models
3. Flexible modeling of complex distributions
4. Efficient learning with small data sets

Current research in Bayesian methods focuses on:

1. Scalable algorithms for large-scale data
2. Non-conjugate Bayesian models and approximate inference techniques
3. Bayesian optimization for hyperparameter tuning and function optimization
4. Bayesian deep learning for complex data analysis.
Transformers in Machine Learning are a type of model architecture, introduced in the paper "Attention is All You Need" by Vaswani et al. They use self-attention mechanisms to focus on relevant parts of the input sequence for each output 



### Завдання 3: Використання агента для автоматизації процесів
Створіть агента, який допоможе автоматично шукати інформацію про останні наукові публікації в різних галузях. Наприклад, агент має знайти 5 останніх публікацій на тему штучного інтелекту.

**Кроки:**
1. Налаштуйте агента типу ReAct в LangChain для виконання автоматичних запитів.
2. Створіть промпт, який спрямовує агента шукати інформацію в інтернеті або в базах даних наукових публікацій.
3. Агент повинен видати список публікацій, кожна з яких містить назву, авторів і короткий опис.

Для взаємодії з пошуком там необхідно створити `Tool`. В лекції ми використовували `serpapi`. Можна продовжити користуватись ним, або обрати інше АРІ для пошуку (вони в тому числі є безкоштовні). Перелік різних АРІ, доступних в langchain, і орієнтир по вартості запитів можна знайти в окремому документі [тут](https://hannapylieva.notion.site/API-12994835849480a69b2adf2b8441cbb3?pvs=4).

Лишаю також нижче приклад використання одного з безкоштовних пошукових АРІ - DuckDuckGo (не потребує створення токена!)  - можливо він вам сподобається :)


#### `Mistral` and `DuckDuckGoSearchRun`

In [45]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    temperature=overal_temperature
)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [20]:
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description=("Useful for searching information in the internet. "
                     "Use this to search for the latest publications on a given topic.")
    )
]

In [21]:
prompt = hub.pull('hwchase17/react')

d:\anaconda3\envs\py312\Lib\site-packages\langsmith\client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [42]:
agent = create_react_agent(llm, tools, prompt, stop_sequence=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
response = agent_executor.invoke(
    {"input": "Find the 5 latest publications on Artificial Intelligence. "
              "For search you could use scientific papers, news, blogs and other resources. "
              "For each, provide the title, authors (if available), and a short description."})

print(response)



> Entering new AgentExecutor chain...


d:\anaconda3\envs\py312\Lib\site-packages\huggingface_hub\inference\_client.py:2252: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


 I need to search for the latest publications on Artificial Intelligence. I will use the Search tool for this.
Action: Search
Action Input: 'latest publications on Artificial Intelligence'
ObservationThis research work explores the integration of machine learning (ML) and artificial intelligence (AI) in the field of Earth observation (EO). As satellite data continues to grow, offering global insights into environmental processes, there is a pressing need for advanced processing techniques to convert raw data into actionable information. Earth observation (EO), aiming at monitoring the state of planet Earth using remote sensing data, is critical for improving our daily lives and living environment. With a growing number of satellites in orbit, an increasing number of datasets with diverse sensors and research domains are being published to facilitate the research of the remote sensing community. This paper presents a ... This study provides a comprehensive bibliometric and in-depth anal

d:\anaconda3\envs\py312\Lib\site-packages\huggingface_hub\inference\_client.py:2252: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


 I have found 4 publications on Artificial Intelligence. I will find the 5th one.
Action: Search
Action Input: 'latest publications on Artificial Intelligence'
ObservationThis research work explores the integration of machine learning (ML) and artificial intelligence (AI) in the field of Earth observation (EO). As satellite data continues to grow, offering global insights into environmental processes, there is a pressing need for advanced processing techniques to convert raw data into actionable information. Earth observation (EO), aiming at monitoring the state of planet Earth using remote sensing data, is critical for improving our daily lives and living environment. With a growing number of satellites in orbit, an increasing number of datasets with diverse sensors and research domains are being published to facilitate the research of the remote sensing community. This paper presents a ... This study provides a comprehensive bibliometric and in-depth analysis of artificial intelligen

d:\anaconda3\envs\py312\Lib\site-packages\huggingface_hub\inference\_client.py:2252: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


 I have found 5 publications on Artificial Intelligence. I will now provide the final answer.
Final Answer:
1. Title: Advances in Artificial Intelligence and Earth Observation for Urban Studies
   Authors: Not provided
   Description: This commentary discusses the impact of the integration of artificial intelligence (AI) and Earth observation (EO) on urban studies, offering advancements in urban research.

2. Title: A Comprehensive Bibliometric and In-depth Analysis of Artificial Intelligence and Machine Learning Applications in Environmental Monitoring
   Authors: Not provided
   Description: This study provides a comprehensive analysis of AI and ML applications in environmental monitoring, highlighting a notable increase in publications and citations since 2010.

3. Title: Integration of Machine Learning and Artificial Intelligence in Earth Observation
   Authors: Not provided
   Description: This research work explores the integration of ML and AI in the field of Earth observation, 

#### `gpt-4o-mini` and `GoogleSearch`

In [ ]:
llm_gpt = ChatOpenAI(model="gpt-4o-mini", temperature=overal_temperature)

search = GoogleSearchAPIWrapper()


def top5_results(query):
    return search.results(query, 5)


tool = [Tool(
    name="Google Search Snippets",
    description="Search Google for recent results.",
    func=top5_results,
)]

agent = create_react_agent(llm_gpt, tool, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tool, verbose=True)

response = agent_executor.invoke(
    {"input": "Find the 5 latest publications about Artificial Intelligence for 2025. "
              "This could include news, blogs, scientific papers, and other resources. "
              "For each, provide the title, authors (if available), and a short description."})

print(response)



> Entering new AgentExecutor chain...
I need to find the latest publications about Artificial Intelligence from the year 2025. Since it's currently 2023, I will search for any upcoming or anticipated publications, news articles, or scientific papers that may have been announced or are expected to be released in 2025. 

Action: Google Search Snippets  
Action Input: "latest publications about Artificial Intelligence 2025"  [{'title': 'UNESCO dedicates the International Day of Education 2025 to ...', 'link': 'https://www.unesco.org/en/articles/unesco-dedicates-international-day-education-2025-artificial-intelligence', 'snippet': 'Feb 3, 2025 ... Find the latest ideas, publications and reports. ... UNESCO dedicates the International Day of Education 2025 to Artificial Intelligence.'}, {'title': 'Artificial Intelligence', 'link': 'https://arxiv.org/list/cs.AI/recent', 'snippet': 'Journal-ref: IEEE Internet of Things Magazine, 2025. Subjects: Artificial Intelligence (cs.AI). [12] arXiv:25



### Завдання 4: Створення агента-помічника для вирішення бізнес-задач

Створіть агента, який допомагає вирішувати задачі бізнес-аналітики. Агент має допомогти користувачу створити прогноз по продажам на наступний рік враховуючи рівень інфляції і погодні умови. Агент має вміти використовувати Python і ходити в інтернет аби отримати актуальні дані.

**Кроки:**
1. Налаштуйте агента, який працюватиме з аналітичними даними, заданими текстом. Користувач пише

```
Ми експортуємо апельсини з Бразилії. В 2021 експортували 200т, в 2022 - 190т, в 2023 - 210т, в 2024 який ще не закінчився - 220т. Зроби оцінку скільки ми зможемо експортувати апельсинів в 2025 враховуючи погодні умови в Бразилії і попит на апельсини в світі виходячи з економічної ситуації.
```

2. Створіть запит до агента, що містить чітке завдання – видати результат бізнес аналізу або написати, що він не може цього зробити і запит користувача (просто може бути все одним повідомлленням).

3. Запустіть агента і проаналізуйте результати. Що можна покращити?


#### `Mistral` and `DuckDuckGoSearchRun`

In [50]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    temperature=overal_temperature
)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [64]:
search_tool = DuckDuckGoSearchResults(
    name="web_search",
    description=("Useful for when you need to answer questions about current or future  events. "
                 "Use this to search for the economic and weather forecasting")
)

In [65]:
python_tool = PythonREPLTool(
    name="python_interpreter",
    description=("A Python shell. Use this to execute python commands. "
                 "Input should be a valid python command. "
                 "If you want to see the output of a value, "
                 "you necessarily should print it out with `print(...)`. "
                 "Otherwise you won't see the result! It's very important."
                 "For calculate the average increase use difference between year's sales")
)

In [66]:
agent = initialize_agent(
    tools=[search_tool, python_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
user_query = """We export oranges from Brazil.
In 2021, we exported 200 tons, in 2022 - 190 tons, in 2023 - 210 tons,
in 2024, which has not yet ended - 220 tons.
Estimate how many oranges we will be able to export in 2025,
taking into account weather conditions in Brazil and the demand for oranges in the world
based on the economic situation.
At first, estimate economic and weather forecast for 2025 year and only after that make prediction.
Provide the result of the business analysis or write that you cannot do this"""

In [72]:
response = agent.run(user_query)
print(response)



> Entering new AgentExecutor chain...


d:\anaconda3\envs\py312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


 I need to find the economic and weather forecast for 2025. Then I can estimate the orange export for 2025 based on the trends from the past years and the forecast.
Action: web_search
Action Input: Economic and weather forecast for 2025
Observation: snippet: The forecast for 2025 is broadly unchanged from that in the October 2024 World Economic Outlook (WEO), primarily on account of an upward revision in the United States offsetting downward revisions in other major economies. Global headline inflation is expected to decline to 4.2 percent in 2025 and to 3.5 percent in 2026, converging back to ..., title: World Economic Outlook Update, January 2025: Global Growth ... - IMF, link: https://www.imf.org/en/Publications/WEO/Issues/2025/01/17/world-economic-outlook-update-january-2025, snippet: Global economic growth remained resilient in 2024. However, recent indicators suggest softening growth prospects, with measures of economic policy uncertainty having risen markedly alongside the impos

d:\anaconda3\envs\py312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


 I have the economic and weather forecast for 2025. Now I can estimate the orange export for 2025.
Action: python_interpreter
Action Input:
```
import statistics
sales = [200, 190, 210, 220]
average_increase = (220 - 200) / (len(sales) - 1)
predicted_sales = 220 + average_increase
print(predicted_sales)
```
Observation: 226.66666666666666

Thought:

d:\anaconda3\envs\py312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


 I now know the final answer
Final Answer: Based on the trends from the past years and the forecast for 2025, it is estimated that we will be able to export 227 tons of oranges in 2025.

> Finished chain.
Based on the trends from the past years and the forecast for 2025, it is estimated that we will be able to export 227 tons of oranges in 2025.


#### `gpt-4o-mini` and `GoogleSearch`

In [45]:
llm_gpt = ChatOpenAI(model="gpt-4o-mini", temperature=overal_temperature)

search = GoogleSearchAPIWrapper()

search_tool = Tool(
    name="Google Search Snippets",
    description="Search Google for recent results.",
    func=search.run,
)

python_tool = PythonREPLTool(
    name="python_interpreter",
    description=("A Python shell. Use this to execute python commands. "
                 "Input should be a valid python command. "
                 "If you want to see the output of a value, "
                 "you necessarily should print it out with `print(...)`. "
                 "Otherwise you won't see the result! It's very important."
                 "For calculate the average increase use difference between year's sales")
)

agent = create_react_agent(llm_gpt, [search_tool, python_tool], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[search_tool, python_tool], verbose=True)

user_query = """We export oranges from Brazil.
In 2021, we exported 200 tons, in 2022 - 190 tons, in 2023 - 210 tons,
in 2024, which has not yet ended - 220 tons.
Estimate how many oranges we will be able to export in 2025,
taking into account weather conditions in Brazil and the demand for oranges in the world
based on the economic situation.
At first, estimate economic and weather forecast for 2025 year and only after that make prediction.
Provide the result of the business analysis or write that you cannot do this"""

response = agent_executor.invoke(
    {"input": user_query})

print(response)



> Entering new AgentExecutor chain...
To estimate the export of oranges from Brazil in 2025, I need to first gather information on the economic and weather forecasts for that year. This will help in understanding the potential demand and supply conditions. 

Action: Google Search Snippets  
Action Input: "Brazil orange export forecast 2025 economic weather conditions"  Jan 2, 2025 ... Consumption is up with projected higher imports from Brazil and exports are forecast down with lower production. Global Orange Juice Production. 2 days ago ... F = Forecast. Note: Florida orange marketing year starts in October and ends in September of the following year. Source: USDA, Economic Research ... Dec 20, 2024 ... Total fresh orange exports for MY 2024/25 are projected at virtually zero, based on information from. SECEX. Brazil has limited market access to ... Dec 21, 2021 ... The Brazilian orange crop for Marketing Year (MY) 2021/22 is forecast at 405 million 40.8-kg boxes (MBx), equal to 16.

Python REPL can execute arbitrary code. Use with caution.


The search results indicate that Brazil's orange exports for the marketing year 2024/25 are projected to be significantly impacted by lower production, with total fresh orange exports forecasted at virtually zero. This suggests a challenging environment for orange exports due to unfavorable weather conditions and possibly lower economic growth. 

Given that the economic growth is projected to be between 4% and 4.5%, there may be some demand for oranges, but the supply constraints due to lower production will likely limit export capabilities.

Next, I will analyze the historical export data to estimate the potential export for 2025 based on the trends observed in previous years.

Action: python_interpreter  
Action Input: 
# Historical export data
exports = {
    2021: 200,  # tons
    2022: 190,  # tons
    2023: 210,  # tons
    2024: 220   # tons
}

# Calculate the average increase in exports from 2021 to 2024
years = list(exports.keys())
increases = [exports[years[i]] - exports[year

- `Mistral` retrieves economic and weather forecasts but doesn't incorporate them into its predictions. It calculates trends using a simple average of the first and last years. Attempts to refine the query didn't significantly improve aggregation.
- In contrast, `gpt-4o-mini` more accurately calculates the average increase in exports and integrates weather and economic forecasts into the final prediction. 